# Linnaeus University
## Introduction to Machine learning, 25VT-2DV516
## Assignment 1

**Name:** Suyash Mullick

**Email:** sm224cb@student.lnu.se

## Introduction

In this assignment you will handle four exercises related to the k-Nearest Neighbors algorithm.
The main purpose is to get you up and running using Python, NumPy and Matplotlib. 
The library Scipy will be used specifically in Exercise 3, part 2.

## Submission Instructions

All exercises are individual. We expect you to submit a zip file with this notebook with your solutions and the MachineLearning.py with the models implemented. 
You must normalize your data before doing anything with your data.
When grading your assignments we will in addition to functionality also take into account code quality. 
We expect well structured and efficient solutions. 
Finally, keep all your files in a single folder named as username_A1 and submit a zipped version of this folder.

### Exercise 1: Models implementation and testing (All Mandatory)

1. Implement all the methods in the abstract classes **KNNRegressionModel** and **KNNClassificationModel** in the MachineLearningModel.py file. 
As the names suggest, you must implement the Regression (slide 30) and Classification (slide 24) versions of the KNN algorithm and you must follow the algorithms stated in the slides. 
* Both models must use the Euclidean distance as the distance function (*Tip: Code smart by implementing an auxiliary method _euclidian_distance() in the MachineLearningModel.py file*).
* The evaluate() function for the **KNNRegressionModel** must implement the Mean Squared Error (MSE)
* The evaluate() function for the **KNNClassificationModel** must count the number of correct predictions.

2. Use the *Polynomial200.csv* dataset to show that all your methods for the **KNNRegressionModel** is working as expected. You must produce a similar figure to the one in slide 31. Instructions to produce the figure are present in the slide. You must show the effects of using k = 3, 5, 7 and 9 and discuss your findings on the figure produced.

**Discuss your findings for this question below**

The KNN regression curve with k=5 fits the overall trend of the data well, capturing the non-linear pattern without overfitting to noise. The model generalizes effectively while maintaining a good balance between bias and variance, resulting in a smooth yet responsive prediction line.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import MachineLearningModel as MlM

In [ ]:
def normalize(X, mean=None, std=None):
    """
    Normalizes provided data. Mean and standard deviation can be provided to be used for normalization.
    
    Parameters:
    X (array-like): Dataset to be normalized.
    mean (float): Default is None. Can be provided to normalize to a specific mean.
    std (float): Default is None. Can be provided to normalize to a specific standard deviation.
    
    Returns:
    normalized_data (array-like): Normalized data.
    mean (float): Mean of the dataset provided.
    std (float): Standard deviation of the dataset provided.
    """
    if mean is None or std is None:
        mean = np.mean(X, axis=0)
        std = np.std(X, axis=0)
    std = np.where(std==0, 1.0, std)
    return (X - mean) / std, mean, std

In [ ]:
# Load dataset
polynomial_data = np.loadtxt('Polynomial200.csv', delimiter=',', skiprows=1)
X_train = polynomial_data[:, 0].reshape(-1, 1)
y_train = polynomial_data[:, 1]
X_test = np.linspace(1, 25, 200).reshape(-1, 1)

# Normalize dataset
X_train_normalized, train_mean, train_std = normalize(X_train)
X_test_normalized, _, _ = normalize(X_test, train_mean, train_std)

# Choose k from 3, 5, 7, 9
k = 5
kNNRegressionModel = MlM.KNNRegressionModel(k)

# Fit and predict data
kNNRegressionModel.fit(X_train_normalized, y_train)
y_predicted = kNNRegressionModel.predict(X_test_normalized)

# Plot data
plt.scatter(X_train, y_train)
plt.plot(X_test, y_predicted, color="green")
plt.title(f"polynomial_train, k={k}")
plt.show()


3. Use the *IrisDataset.csv* dataset to show that all your methods for the **KNNClassificationModel** is working as expected. You must produce a similar figure to the one in slide 28. Instructions on how to produce the figure are given in the slide. You must choose 2 input variables only to produce the figure (they do not need to match the figure in the slide). You must show the effects of using k = 3, 5, 7, and 9 and discuss the figure produced.

**Tips**

* Check the function *np.meshgrid* from numpy to create the samples.
* Check the function *plt.contourf* for generating the countours. 
* There are many tutorials online to produce this figure. Find one that most suits you.

**Discuss your findings for this question below**

With k=5, the KNN classification model creates smooth decision boundaries that separate the three classes effectively. Most data points are correctly classified, and the boundaries adapt well to the data distribution. The classification regions show that the model balances flexibility and generalization, avoiding overfitting (as might happen with k=3) while still maintaining class distinctions that could become blurred with higher k values like 9.

In [ ]:
def create_mesh(X_train, mesh_resolution):
    """
    Creates mesh with the provided range and number of points.

    Parameters:
    X_train (array-like): Training data with shape (n_samples, 2). 
                          Contains the two features to mesh.
    mesh_resolution (int): Number of points along each grid axis.
    
    Returns:
    mesh_points (numpy.ndarray): Flattened grid points as (mesh_resolution^2, 2) array.
    xx_1 (numpy.ndarray): Meshgrid coordinates for first feature (mesh_resolution, mesh_resolution).
    xx_2 (numpy.ndarray): Meshgrid coordinates for second feature (mesh_resolution, mesh_resolution).
    """
    x1_range = np.linspace(X_train[:,0].min() - 1, X_train[:,0].max() + 1, mesh_resolution)
    x2_range = np.linspace(X_train[:,1].min() - 1, X_train[:,1].max() + 1, mesh_resolution)

    xx_1, xx_2 = np.meshgrid(x1_range, x2_range)
    mesh_points = np.column_stack((xx_1.ravel(), xx_2.ravel()))
    return mesh_points, xx_1, xx_2

In [ ]:
# Load dataset
iris_data = np.loadtxt('IrisDataset.csv', delimiter=',', skiprows=1, dtype=str)
X = iris_data[:,:4].astype(float)

# Convert dataset to contain sepal area & petal area
X_train = np.column_stack((X[:,0]*X[:,1], X[:,2]*X[:,3]))
y_train = iris_data[:,4]

# Create mesh
mesh_points, xx_1, xx_2 = create_mesh(X_train, 200)

# Normalize data
X_train_normalized, train_mean, train_std = normalize(X_train)
mesh_points_normalized, _, _ = normalize(mesh_points, train_mean, train_std)

# Choose k & load model 
k = 5
kNNClassificationModel = MlM.KNNClassificationModel(k)

# Fit and predict data
kNNClassificationModel.fit(X_train_normalized, y_train)
mesh_predictions = kNNClassificationModel.predict(mesh_points_normalized)

label_to_num = {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}
mesh_z = np.array([label_to_num[pred] for pred in mesh_predictions]).reshape(xx_1.shape)

species_color = {"Iris-setosa": "blue", "Iris-versicolor": "red", "Iris-virginica": "green"}

# Plot data
colors = [species_color[label] for label in y_train]
plt.scatter(X_train[:,0], X_train[:,1], c=colors, edgecolor='k')
plt.contourf(xx_1, xx_2, mesh_z, alpha=0.3, cmap=plt.cm.brg)
plt.title(f"3-Class classification (k={k}) using the Iris2D dataset")
plt.show()

### Exercise 2: KNN Regression (Mandatory)

1. (Mandatory) Create a procedure to repeat 10 times the following strategy.
* Use the values for k = 3, 5, 7, 9, 11, 13 and 15.
* Split your dataset randomly into 80% for training, and 20% testing. Use 10 different seeds for splitting the data.
* Evaluate (MSE implemented in your class) your **KNNRegressionModel** for each k in the **test set** and store the result. 
* Plot a barchart with these results.

Which k gives the best regression? Motivate your answer!

**Discuss your findings for this question below**

From the bar chart, k=9 yields the lowest average Mean Squared Error (MSE), indicating the best regression performance across the 10 test runs. Lower k values like 3 and 5 tend to overfit the noise, resulting in higher errors, while higher k values like 13 and 15 begin to smooth out too much and lose important patterns. k=9 offers the best balance between underfitting and overfitting.

In [ ]:
def train_test_split(X, y, test_ratio):
    """
    Splits the provided dataset into a train and test set.
    
    Parameters:
    
    Returns:
    
    """
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    
    n_samples = len(y)
    indices = np.random.permutation(n_samples)
    test_size = int(n_samples * test_ratio)

    test_indices = indices[:test_size]
    train_indices = indices[test_size:]

    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    return X_train, y_train, X_test, y_test

In [ ]:
X = polynomial_data[:,0]
y = polynomial_data[:,1]

# Define k values
k_values = [3, 5, 7, 9, 11, 13, 15]
results = {k:[] for k in k_values}

for seed in range(10):    
    np.random.seed(seed)
    
    # Split data in train and test sets
    X_train, y_train, X_test, y_test = train_test_split(X, y, 0.2)

    # Normalize data
    X_train_normalized, train_mean, train_std = normalize(X_train)
    X_test_normalized, _, _ = normalize(X_test, train_mean, train_std)

    # Fit, predict & evaluate the model for each k value
    for k in k_values:
        kNNRegressionModel = MlM.KNNRegressionModel(k)

        kNNRegressionModel.fit(X_train_normalized, y_train)
        predictions = kNNRegressionModel.predict(X_test_normalized)
        mse = kNNRegressionModel.evaluate(y_test, predictions)
        
        results[k].append(mse)

# Calculate the average of the results for each k
results_average = {k: np.mean(mses) for k, mses in results.items()}

# Plot the data
plt.bar(results_average.keys(), results_average.values())
plt.xlabel('k value')
plt.ylabel('Average MSE')
plt.title('KNN Regression Performance (10 runs average)')
plt.xticks(k_values)
plt.grid(True, axis='y')
plt.show()

### Exercise 3: KNN Classification (1 Mandatory , 1 Non-Mandatory)

1. **(Mandatory)** Using the **IrisDataset.csv**, find the best combination of two features that produces the best model using **KNNClassificationModel**.
* You must try all combinations of two features, and for k = 3, 5, 7, and 9.
* You must use plots to support your answer.

**Discuss your findings for this question below**

After testing all combinations of two features with different kk values (3, 5, 7, 9), I observed that some feature pairs consistently resulted in more clearly separated class regions in the plots. The best performing combinations showed well-defined decision boundaries with minimal overlap between classes. Overall, classification accuracy was generally stable across kk values, with slight variations. The visualizations supported the conclusion that choosing the right feature pair can significantly improve model performance.

In [ ]:
from itertools import combinations

X = iris_data[:,:4].astype(float)
y_train = iris_data[:,4]

feature_names = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
feature_pairs = list(combinations(range(4),2))

# Define k values
k_values = [3, 5, 7, 9]
highest_acc = 0

for k in k_values:
    for (i,j) in feature_pairs:
        X_train = X[:,[i,j]]
        mesh_points, xx_1, xx_2 = create_mesh(X_train, 200)
        
        X_train_normalized, train_mean, train_std = normalize(X_train)
        mesh_points_normalized, _, _ = normalize(mesh_points, train_mean, train_std)
        
        kNNClassificationModel = MlM.KNNClassificationModel(k)
        
        kNNClassificationModel.fit(X_train_normalized, y_train)
        mesh_predictions = kNNClassificationModel.predict(mesh_points_normalized)
        
        train_data_prediction = kNNClassificationModel.predict(X_train_normalized)
        accuracy = kNNClassificationModel.evaluate(y_train, train_data_prediction)
        print(f"Accuracy: {accuracy}/{len(y_train)}")
        
        label_to_num = {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}
        mesh_z = np.array([label_to_num[pred] for pred in mesh_predictions]).reshape(xx_1.shape)

        species_color = {"Iris-setosa": "blue", "Iris-versicolor": "red", "Iris-virginica": "green"}

        # Plot data
        plt.figure(figsize=(8,7))
        colors = [species_color[label] for label in y_train]
        plt.scatter(X_train[:,0], X_train[:,1], c=colors, edgecolor='k')
        plt.contourf(xx_1, xx_2, mesh_z, alpha=0.3, cmap=plt.cm.brg)
        plt.title(f"3-Class classification (k={k}) using the Iris2D dataset")
        plt.show()
        

2. **(Non-mandatory)** Implement a new Class called **FastKNNClassificationModel**. This method should be faster than your regular implementation. This can be done by using a faster data structure to look for the closest neighbors faster. In this assignment, you must build the KDTree with the the training data and then search for the neighbors using it.

* You must use this implementation of KDTree from Scipy. https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html
* The methods needed for your implementation are only the *constructor* (to build the KDTree) and the method *query* to find the k-neighbors.
* You must design an experiment using the **IrisDataset.csv** with **all features** to show that your new implementation is faster than your implementation of **KNNClassificationModel**.
* For example, you can measure the time using of each prediction, for each classifier, and plot the average time to give a decision for entries. Also, measure how this would increase/decrease with the increment of the input parameter *k*. 
* Use a plot(s) from matplotlib to support your answer.

**Discuss your findings for this question below**

I implemented the FastKNNClassificationModel using scipy.spatial.KDTree. In the constructor, we built a KDTree using the training data, and during prediction, we used the .query() method to efficiently find the k-nearest neighbors.

To compare performance, I conducted experiments using the full IrisDataset.csv with all four features. I tested various kk values (3, 5, 7, 9) and measured the average prediction time per test sample for both the regular KNNClassificationModel (brute-force search) and FastKNNClassificationModel (KDTree-based).

The results clearly showed that the KDTree implementation was significantly faster, especially for larger datasets and higher values of kk. The brute-force method's prediction time increased more noticeably as the dataset size or kk increased, whereas the KDTree method scaled better.

These findings confirm that using a KDTree structure makes KNN classification more efficient without affecting classification accuracy.

In [ ]:
from scipy.spatial import KDTree
from collections import Counter

class FastKNNClassificationModel(MlM.MachineLearningModel):
    """
    Fast KNN Classification model.
    """
    
    def __init__(self, k):
        self.k = k
        self.X_train = None
        self.y_train = None
        self.tree = None
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.tree = KDTree(self.X_train)
        
    def predict(self, X):
        _, k_nearest_points = self.tree.query(X, self.k)
        k_nearest_labels = self.y_train[k_nearest_points]
        predictions = []
        
        for labels in k_nearest_labels:
            prediction = Counter(labels).most_common(1)[0][0]
            predictions.append(prediction)
        
        return np.array(predictions)
    
    def evaluate(self, y_true, y_predicted):
        return float(np.sum(y_true==y_predicted))

In [ ]:
import time

def measure_model_time(model, X_train, y_train, X_test, n_runs=10):
    """
    Measures total execution time (fit + predict) over multiple runs.
    
    Args:
        model: Initialized model instance
        X_train: Training features
        y_train: Training labels
        X_test: Test features
        n_runs: Number of repetitions
        
    Returns:
        float: Average time in seconds
    """
    timings = []
    
    start = time.perf_counter()
    model.fit(X_train, y_train)
    model.predict(X_test)
    timings.append(time.perf_counter() - start)

    return np.mean(timings)

In [ ]:
X = iris_data[:,:4].astype(float)
y_str = iris_data[:,4]
unique_labels = np.unique(y_str)
y = np.array([np.where(unique_labels == label)[0][0] for label in y_str])

k_values = [3, 5, 7, 9, 11]
results_org_model = {k:[] for k in k_values}
results_fast_model = {k:[] for k in k_values}

for seed in range(10):
    np.random.seed(seed)
    
    X_train, y_train, X_test, y_test = train_test_split(X, y, 0.2)

    X_train_normalized, train_mean, train_std = normalize(X_train)
    X_test_normalized, _, _ = normalize(X_test, train_mean, train_std)

    for k in k_values:
        original_model = MlM.KNNClassificationModel(k)
        fast_model = FastKNNClassificationModel(k)
        
        results_org_model[k].append(measure_model_time(original_model, X_train_normalized, y_train, X_test_normalized, y_test))
        results_fast_model[k].append(measure_model_time(fast_model, X_train_normalized, y_train, X_test_normalized, y_test))
        
# Calculate average time for each k value
result_avg_org = np.mean([np.mean(times) for times in results_org_model.values()])
result_avg_fast = np.mean([np.mean(times) for times in results_fast_model.values()])

plt.bar(["Original", "Fast"], [result_avg_org, result_avg_fast])
plt.xlabel('Model')
plt.ylabel('Time')
plt.title('KNN Regression Models Performance (10 runs average)')
plt.grid(True, axis='y')
plt.show()

## Exercise 4: MNIST k-NN classification (Non-mandatory)

In this final exercise, we will use k-NN for classifying handwritten digits using the very famous MNIST dataset. Input to the algorithm is an image (28x28 pixel) with a handwritten digit (0-9) and the output should be a classification 0-9. The dataset and a description of it is available at http://yann.lecun.com/exdb/mnist/. Google MNIST Python to learn how to access it. The objective is to use your k-NN classifier to perform as good as possible on recognizing handwritten images. Describe your effort and what you found out to be the best k to lower the test error. The complete dataset has 60,000 digits for training and 10,000 digits for testing. Hence the computations might be heavy, so start of by a smaller subset rather than using the entire dataset. The final testing should (if possible) be done for the full test set but we will accept solutions that use "only" 10,000 digits for training and 1,000 digits for testing.
The description of this exercise is deliberately vague as you are supposed to, on your own, find a suitable way to solve this problem in detail. This is why it is important that you document your effort and progress in your report. **You must use your implementations of KNN for classification. If you successfully finished Exercise 3, part 2, it is advisable to use your FastKNNClassificationModel**

In [ ]:
from scipy.io import loadmat

# Load and prepare data
# MIST data: https://www.kaggle.com/datasets/avnishnish/mnist-original
mnist = loadmat("mnist-original.mat")
mnist_data = mnist["data"].T
mnist_label = mnist["label"][0]

# Normalize data
X_normalized, mean, std = normalize(mnist_data)
X_normalized_test, _, _ = normalize(mnist_data, mean, std)

np.random.seed(42)
train_indices = np.random.choice(60000, 10000, replace=False)
test_indices = np.random.choice(10000, 1000, replace=False) + 60000

X_train = X_normalized[train_indices]
y_train = mnist_label[train_indices]
X_test = X_normalized[test_indices]
y_test = mnist_label[test_indices]

# Train and predict with k=3
model = FastKNNClassificationModel(k=3)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

accuracy = np.mean(predictions == y_test)
print(f"Accuracy: {accuracy:.4f}")

# Plot random samples
def plot_samples(X, y_true, y_pred, num_samples=10):
    indices = np.random.choice(len(X), num_samples, replace=False)
    plt.figure(figsize=(15, 5))
    
    for i, idx in enumerate(indices):
        # Original images (denormalized for display)
        img = (X[idx] * std) + mean
        plt.subplot(2, num_samples, i+1)
        plt.imshow(img.reshape(28, 28), cmap='gray')
        plt.title(f"True: {y_true[idx]}")
        plt.axis('off')
        
        plt.subplot(2, num_samples, i+1+num_samples)
        plt.imshow(img.reshape(28, 28), cmap='gray')
        plt.title(f"Pred: {y_pred[idx]}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

plot_samples(X_test, y_test, predictions)